In [ ]:
import os, shutil
train_dir = '../train'
validation_dir = '../validation'
test_dir = '../test'

In [ ]:
from keras.utils import image_dataset_from_directory

IMG_SIZE = 150

train_dataset = image_dataset_from_directory(train_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
validation_dataset = image_dataset_from_directory(validation_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))
test_dataset = image_dataset_from_directory(test_dir, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE))

#### Dividir o dataset ####
def get_dataset_size(dataset):
    return sum(1 for _ in dataset)

train_size = get_dataset_size(train_dataset)
validation_size = get_dataset_size(validation_dataset)

part_train_size = train_size // 6
part_validation_size = validation_size // 6

def split_dataset(dataset, part_size):
    parts = []
    for i in range(6):
        parts.append(dataset.skip(i * part_size).take(part_size))
    return parts

train_parts = split_dataset(train_dataset, part_train_size)
validation_parts = split_dataset(validation_dataset, part_validation_size)

train_dataset_1, train_dataset_2, train_dataset_3, train_dataset_4, train_dataset_5, train_dataset_6 = train_parts
validation_dataset_1, validation_dataset_2, validation_dataset_3, validation_dataset_4, validation_dataset_5, validation_dataset_6 = validation_parts

print(f"Train dataset parts sizes: {[get_dataset_size(part) for part in train_parts]}")
print(f"Validation dataset parts sizes: {[get_dataset_size(part) for part in validation_parts]}")


In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

def visualize_images(dataset, num_images=9):
    
    dataset_iter = iter(dataset)
    
    images, labels = next(dataset_iter)
    
    plt.figure(figsize=(10, 10))
    for i in range(num_images):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8")) 
        plt.title(f"Label: {labels[i].numpy()}")
        plt.axis("off")
    plt.show()

# Exemplo de uso da funcao acima para visualizar uma imagem por cada um dos 6 datasets
visualize_images(train_dataset_1, num_images=1)
visualize_images(train_dataset_2, num_images=1)
visualize_images(train_dataset_3, num_images=1)
visualize_images(train_dataset_4, num_images=1)
visualize_images(train_dataset_5, num_images=1)
visualize_images(train_dataset_6, num_images=1)


In [ ]:
from keras.applications.vgg19 import VGG19

conv_base = VGG19(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3)) 
conv_base.trainable = True

# Deixar todas as camadas, exceto as últimas cinco, não treináveis (congeladas)
for layer in conv_base.layers[:-5]: 
    layer.trainable = False

In [ ]:
from tensorflow.keras.models import Model

# Função para imprimir o estado das camadas de forma recursiva
def print_layer_trainable_status(layer, indent=0):
    print(f"{' ' * indent}Layer: {layer.name}, Trainable: {layer.trainable}")
    if isinstance(layer, Model):
        for sub_layer in layer.layers:
            print_layer_trainable_status(sub_layer, indent + 2)

print_layer_trainable_status(conv_base)

In [ ]:
from tensorflow import keras

#Adicionar o modelo treinado com feature extraction
base_model = keras.models.load_model('models/ModelT_transferLearning_featureExtraction_WithoutDataAumentation_OnlyClassification.h5')

base_model.summary()

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import GaussianNoise

# Definir a sequência de data augmentation
data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
        layers.RandomContrast(0.2),
        GaussianNoise(stddev=0.1),
    ]
)

# Criar uma camada de input com o mesmo shape da VGG19
input_layer = Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# Aplicar data augmentation à camada de input
augmented_input = data_augmentation(input_layer)

# Passar a camada de data_augmentation como input para o modelo base VGG19
vgg19_output = conv_base(augmented_input)

# Passar a saída da VGG19 para o modelo pré-treinado
model_output = base_model(vgg19_output)

# Criar o novo modelo combinado com a VGG19 e o modelo pré-treinado
model = Model(inputs=input_layer, outputs=model_output)

In [ ]:
print_layer_trainable_status(model)

In [ ]:
#A partir deste bloco iremos treinar o modelo para os sub datasets
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint_callback = ModelCheckpoint(
    filepath='models/ModelT_transferLearning_fineTuning_WithDataAumentation_best.h5',
    save_best_only=True,
    monitor='val_loss',
    verbose=1 
)

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

In [ ]:
#Subset 1
history = model.fit(train_dataset_1, epochs=10, validation_data=validation_dataset_1, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 2
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-7, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(train_dataset_2, epochs=5, validation_data=validation_dataset_2, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 3
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-7)

history = model.fit(train_dataset_3, epochs=5, validation_data=validation_dataset_3, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
for layer in conv_base.layers[-10:]:
    layer.trainable = True

In [ ]:
print_layer_trainable_status(model)

In [ ]:
#Subset 3
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(train_dataset_3, epochs=4, validation_data=validation_dataset_3, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 4
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-5, weight_decay=1e-1),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(train_dataset_4, epochs=5, validation_data=validation_dataset_4, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 5
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-6, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-8)

history = model.fit(train_dataset_5, epochs=5, validation_data=validation_dataset_5, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 6
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-7, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-9)

history = model.fit(train_dataset_6, epochs=5, validation_data=validation_dataset_6, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 3
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-8, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-10)

history = model.fit(train_dataset_3, epochs=5, validation_data=validation_dataset_3, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 1
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-9, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-10)

history = model.fit(train_dataset_1, epochs=5, validation_data=validation_dataset_1, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 2
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-10, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-12)

history = model.fit(train_dataset_2, epochs=5, validation_data=validation_dataset_2, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 6))

# Gráfico da Accuracya
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Accuracy no Treino')
plt.plot(epochs, val_acc, 'b-', label='Accuracy na Validação')
plt.title('Accuracy no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))  
plt.yticks(fontsize=12) 
plt.tight_layout()

# Gráfico da Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Loss no Treino')
plt.plot(epochs, val_loss, 'b-', label='Loss na Validação')
plt.title('Loss no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))
plt.yticks(fontsize=12) 
plt.tight_layout()

# Mostrar os gráficos
plt.show()

In [ ]:
for layer in conv_base.layers[-15:]:
    layer.trainable = True

In [ ]:
print_layer_trainable_status(model)

In [ ]:
#Subset 2
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-11, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-12)

history = model.fit(train_dataset_2, epochs=5, validation_data=validation_dataset_2, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
#Subset 4
model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(learning_rate=1e-12, weight_decay=1e-2),metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=1e-14)

history = model.fit(train_dataset_4, epochs=3, validation_data=validation_dataset_4, batch_size=128, callbacks=[checkpoint_callback,early_stopping, reduce_lr])

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.figure(figsize=(14, 6))

# Gráfico da Accuracya
plt.subplot(1, 2, 1)
plt.plot(epochs, acc, 'bo-', label='Accuracy no Treino')
plt.plot(epochs, val_acc, 'b-', label='Accuracy na Validação')
plt.title('Accuracy no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))  
plt.yticks(fontsize=12) 
plt.tight_layout()

# Gráfico da Loss
plt.subplot(1, 2, 2)
plt.plot(epochs, loss, 'bo-', label='Loss no Treino')
plt.plot(epochs, val_loss, 'b-', label='Loss na Validação')
plt.title('Loss no Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()
plt.grid(True) 
plt.xticks(range(1, len(acc) + 1, 2))
plt.yticks(fontsize=12) 
plt.tight_layout()

# Mostrar os gráficos
plt.show()

In [ ]:
from tensorflow import keras

loaded_model = keras.models.load_model('models/ModelT_transferLearning_fineTuning_WithDataAumentation_best.h5')

val_loss, val_acc = loaded_model.evaluate(test_dataset) 
print('val_acc:', val_acc)